In [3]:
## Setting up a custom stylesheet in IJulia
file = open("./../style.css") # A .css file in the same folder as this notebook file
styl = read(file, String) # Read the file
HTML("$styl") # Output as HTML


HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.CodeMirror{\r\n        font-family: \"PT Mono\";\r\n        font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

## juliaGPU (based on the [juliaGPU 2021 workshop](https://www.youtube.com/watch?v=Hz9IMJuW5hU))

We have different packages available at juliaGPU organization [gitHub](https://cuda.juliagpu.org/stable/) but only some of them are useful:

* Infrastructure: `GPUArrays.jl` `Adapt.jl` `GPUComplier.jl`
* Library wrappers: `OpenCL.jl` `VulkanCore.jl` `MetalCore.jl`
* Programming frameworks (useful): `CUDA.jl`, `AMDGPU.jl`,`oneAPI.jl` and `ArrayFire.jl`(C library)  



### Back end 
Analogsly to Julia with `LLVM.jl` a real time complier that generates machine code into the CPU they use `GPUComplier.jl`, then `LLVM.jl` create `LLVM.jl` to generate GPU code.  
### Architecture 
From Host to Device the data is transferred via PCI/Express bus which is actually really slow, thus we want to avoid copying data from CPU to GPU. Then the CPU inside has a lot of CPU SM streaming multi-processors that can access to shared mem (similar to cahce L2), registers (cache L1) or global memory. Occupancy is the Nvidia term of how to manage all the memory available by a program. 
### Implicit parallelism
There are inner functions that are already implemented using parallelism like `CUDA.reduce()`, and we don't have to worry about implementing it. 


<img src=arch.png>

In [1]:
# load CUDA.jl
import Pkg
using CUDA

# check out the CUDA version
CUDA.versioninfo()

CUDA toolkit 11.7, artifact installation
NVIDIA driver 515.57.0, for CUDA 11.7
CUDA driver 11.7

Libraries: 
- CUBLAS: 11.10.1
- CURAND: 10.2.10
- CUFFT: 10.7.2
- CUSOLVER: 11.3.5
- CUSPARSE: 11.7.3
- CUPTI: 17.0.0
- NVML: 11.0.0+515.57
- CUDNN: 8.30.2 (for CUDA 11.5.0)
- CUTENSOR: 1.4.0 (for CUDA 11.5.0)

Toolchain:
- Julia: 1.7.3
- LLVM: 12.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80

1 device:
  0: NVIDIA GeForce RTX 3060 Laptop GPU (sm_86, 5.598 GiB / 6.000 GiB available)


### Options for CUDA selection
There's two special environment variables that can currently be used to influence the CUDA toolkit that's used:

* `JULIA_CUDA_VERSION` that sets the CUDA version that we want to install (this is useful if any update version is not working with our set up) 
* `JULIA_CUDA_USE_BINARYBUILDER` set to false will install 

Besides we have this customization variables is not recommended to change because CUDA.jl will automatically select the correct one. 

### High order abstraction 
As CUBLAS or other libraries are C libraries are limited in terms of which types it supports so we need a generic fallbacks. Although the CuArray type and its support for NVIDIA's vendor libraries is nice, it isn't really novel and could as well have been implemented in Python (and libraries like CuPy do exactly that). The real flexibility of Julia's arrays comes from its higher-order abstractions, where you combine the abstraction with custom code:

In [2]:
a = CUDA.ones(5)
# aply a function to each element of a
y = broadcast(a) do x
    2x
end
# or what is the same 
y = a.*2
# we have map 
y = map(a) do x
    2x
end
# also accumulate and reduce 
red = reduce(+, a)
acc = accumulate(*,a)


5-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 1.0
 1.0
 1.0
 1.0

Here it is important to highlight that CUDA.jl doesn't provides a tensor complier, as consequence we can't merge functions like accumulate(reduce) for instance. If we merge operations then will be launching as many kernels as nesting operations are executed. For instance what happen if we want to sum each column of a CuArray and divide into the number of elements of each column matrix:

In [3]:
a = CUDA.ones(10,10)

y = broadcast(eachcol(a)) do x
    sum(x)/length(eachcol(a))
end
# this will create 10 kernels one per each column of a 
# let's mesure the performance 
using BenchmarkTools 
@benchmark CUDA.@sync y = broadcast(eachcol(a)) do x
                        sum(x)/length(eachcol(a))
                      end


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  147.644 μs …  37.570 ms  ┊ GC (min … max): 0.00% … 37.59%
 Time  (median):     155.161 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   182.644 μs ± 764.426 μs  ┊ GC (mean ± σ):  3.27% ±  0.78%

  ▇█▅▄▄▃▂▂▁▁                                                    ▂
  ████████████▇▇▇▅▄▄▄▃▄▃▁▄▁▄▁▁▃▁▃▃▄▆▆▅▅▁▃▄▃▁▁▃▁▃▁▁▁▁▄▁▁▁▁▁▃▃▄▄▅ █
  148 μs        Histogram: log(frequency) by time        512 μs <

 Memory estimate: 23.39 KiB, allocs estimate: 501.

In [4]:
# another possiblity is to use dims keyword like
sum(a; dims=2) / size(a,2)

using BenchmarkTools 
@benchmark CUDA.@sync sum(a; dims=2) / size(a,2)
# is arround 10 times faster since only one kernel is complied and executed 

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.387 μs … 484.670 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.688 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.060 μs ±  13.778 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▇█▄▂                                                        
  ▂▅█████▇▅▄▃▃▃▂▂▂▂▂▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  12.4 μs         Histogram: frequency by time         35.5 μs <

 Memory estimate: 2.70 KiB, allocs estimate: 57.

### More complicated applications can be faced using Tulio.jl, a tensor compiler

### Reduce example

We might start with a serial simple for implementation for one single thread;

In [5]:
function reduce_singlethread(op, a, b)
    for i in 1:length(a)
        b[] = op(b[], a[i])
    end
    return
end
# define variables
c_a = 1:16
d_a = CuArray(1:16)
d_b = CuArray([0])
# lunch kernel
@cuda(
    threads = 1,
    reduce_singlethread(+, d_a, d_b)
    )
# test the result 
using Test
@test CUDA.@allowscalar d_b[] == sum(c_a)


Test Passed
  Expression: #= In[5]:18 =# CUDA.@allowscalar d_b[] == sum(c_a)

Obviously this not a very efficient implementation since is a serial one (only one thread). We can test it using `@sync` macro to force the CPU to wait until GPU completes its work. Otherwise we will be measuring the time to queue an operation and not the time it takes to finish it. 

In [6]:
N = 2048
@benchmark CUDA.@sync @cuda( 
    threads = 1, 
        reduce_singlethread(+,   $(CUDA.rand(N, N)), $(CUDA.rand(N, N)))
    )

BenchmarkTools.Trial: 17 samples with 1 evaluation.
 Range (min … max):  311.749 ms … 312.385 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     312.064 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   312.069 ms ± 184.298 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █        ▁   ▁        ▁    ▁█ ▁   ▁ ▁ ▁   ▁▁   ▁         ▁  ▁  
  █▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁██▁█▁▁▁█▁█▁█▁▁▁██▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁█ ▁
  312 ms           Histogram: frequency by time          312 ms <

 Memory estimate: 3.42 KiB, allocs estimate: 62.

Other option is to use `CUDA.@atomic` to take advantage of the SMPD paradigm: 

In [7]:
function reduce_atomic(op, a, b)
    index = threadIdx().x + (blockIdx().x -1)*blockDim().x
    @inbounds CUDA.@atomic b[] = b[] + a[index]
    return
end
# define variables
c_a = 1:16
d_a = CuArray(1:16)
d_b = CuArray([0])
# lunch kernel
@cuda(
    threads = 16,
    blocks = 1,
    reduce_singlethread(+, d_a, d_b)
    )
# test the result 
using Test
@test CUDA.@allowscalar d_b[] == sum(c_a)

Test Passed
  Expression: #= In[7]:18 =# CUDA.@allowscalar d_b[] == sum(c_a)

Let's now benchmark our implementation

In [8]:
N = 2048
@benchmark CUDA.@sync @cuda( 
    threads = 1024, 
    blocks = 2,
        reduce_atomic(+,   $(CUDA.rand(N, N)), $(CUDA.rand(N, N)))
    )

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):  7.839 μs … 142.948 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.600 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.244 μs ±   4.011 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▆▇█▇▅▅▄▄▄▄▃▃▂▁▂▁▁                                  ▁      ▂
  ▃████████████████████▇█▇▇▇▅▆▇▇▇▅▆▇▆▆▆▆▅▄▆▄▅▆▇▅▄▄▃▅▃▅▇██▅▅▄▄ █
  7.84 μs      Histogram: log(frequency) by time      16.6 μs <

 Memory estimate: 304 bytes, allocs estimate: 5.

but we can still improving it using a parallel reduce strategy like this: 

<img src=parallel_reduction.jpg>

The main idea is that each thread reduces its flowing element and so on, finally a the value is written in global mem

In [9]:
#Lets start with a single block reduction 

function reduce_block(op, a, b)
    num_elements = blockDim().x*2
    thread = threadIdx().x
    
    #parallel reduction of values in a block (stride or distance between each thread reduction) 
    stride = 1
    # while still have elements to reduce 
    while stride < num_elements
        # add a barrier to sync threads
        sync_threads()
        # compute index to reduce 
        index = 2*stride*(thread - 1) + 1 
        # check index and index + d are inbounds a
        @inbounds if index ≤ num_elements && index + stride ≤ length(a)
            CUDA.@cuprintln ("thread $thread: a[$index] + a[$(index + stride)] = $(a[index] + a[index + stride])")
            a[index] = op(a[index], a[index + stride])
        end
        stride *= 2
        thread == 1 && CUDA.@cuprintln("stride = $stride")
    end
    # download the block reduction to global mem
    if thread == 1 
        b[] = a[1]
    end
    return nothing
end

c_a = 1:16
d_a = CuArray(1:16)
d_b = CuArray([0])
# lunch kernel
@cuda(
    threads = 16,
    blocks = 1,
    reduce_block(+, d_a, d_b)
    )
# test the result 
using Test
@test CUDA.@allowscalar d_b[] == sum(c_a)


thread 1: a[1] + a[2] = 3
thread 2: a[3] + a[4] = 7
thread 3: a[5] + a[6] = 11
thread 4: a[7] + a[8] = 15
thread 5: a[9] + a[10] = 19
thread 6: a[11] + a[12] = 23
thread 7: a[13] + a[14] = 27
thread 8: a[15] + a[16] = 31
stride = 2
thread 1: a[1] + a[3] = 10
thread 2: a[5] + a[7] = 26
thread 3: a[9] + a[11] = 42
thread 4: a[13] + a[15] = 58
stride = 4
thread 1: a[1] + a[5] = 36
thread 2: a[9] + a[13] = 100
stride = 8
thread 1: a[1] + a[9] = 136
stride = 16
stride = 32


Test Passed
  Expression: #= In[9]:41 =# CUDA.@allowscalar d_b[] == sum(c_a)

We want that different threads access to consecutive memory items (coalesced access pattern). Now we can extend the implementation for a general grid with multiple blocks. Now, we do the reduction for each block and then do an atomic operation to a global mem.

In [10]:
function reduce_grid_atomic(op, a, b)
    num_elements = blockDim().x*2
    thread = threadIdx().x
    block = blockIdx().x
    
    #parallel reduction of values in a block (stride or distance between each thread reduction) 
    stride_threads = 1
    # parallel reduction between blocks has a stride of 
    stride_blocks = (block - 1)*num_elements

    
    # while still have elements to reduce 
    while stride_threads < num_elements
        # add a barrier to sync threads
        sync_threads()
        # compute index to reduce 
        index = 2*stride_threads*(thread - 1) + 1 
        # check index and index + d are inbounds a
        @inbounds if index ≤ num_elements && index + stride_threads + stride_blocks ≤ length(a)
#             CUDA.@cuprintln ("thread $thread: a[$index] + a[$(index + stride_blocks)] = $(a[index] + a[index + stride_blocks])")
            a[stride_blocks + index] = op(a[index + stride_blocks], a[index + stride_threads + stride_blocks])
        end
        stride_threads *= 2
    end
    # do attomic operatios with the first entry of ech block (sum through each block)
    if thread == 1 
        CUDA.@atomic b[] = op(b[], a[stride_blocks + 1])
    end
    return nothing
end

# define test inputs
c_a = 1:16
d_a = CuArray(1:16)
d_b = CuArray([0])
# lunch kernel
@cuda(
    threads = 2,
    blocks = 4,
    reduce_grid_atomic(+, d_a, d_b)
    )
# test the result 
using Test
CUDA.@allowscalar d_b
@test CUDA.@allowscalar d_b[] == sum(c_a)

Test Passed
  Expression: #= In[10]:45 =# CUDA.@allowscalar d_b[] == sum(c_a)

In [11]:
N = 2048
@benchmark CUDA.@sync @cuda( 
    threads = 1024, 
    blocks = 2,
        reduce_grid_atomic(+,   $(CUDA.rand(N, N)), $(CUDA.rand(N, N)))
    )

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.207 μs … 127.096 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.829 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.714 μs ±   3.798 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁█▇▆▅▄▄▄▅▅▄▄▃▃▂▂▂▁▁▁                               ▁▁       ▂
  ██████████████████████████▇▇▇▇▇▆▆▆▄▆▆▅▅▅▄▅▅▄▄▅▄▅▅▆▆██▅▆▆▄▆▆ █
  7.21 μs      Histogram: log(frequency) by time      17.1 μs <

 Memory estimate: 304 bytes, allocs estimate: 5.

Another option is to implement the reduce technique using shared memory that each multiprocessor has, the reduce is done into a shared memory variable and finally an atomic operation is used to download the data to global memory. The shared memory is used as a cache memory and also to communicate between threads. We can use shared memory as a buffer (we should avoid here bank conflicts (at the same operation threads access to the same bank slot).

In [12]:
function reduce_grid_shared(op, a::AbstractArray{T}, b) where {T}
    num_elements = blockDim().x*2
    thread = threadIdx().x
    block = blockIdx().x
    #parallel reduction of values in a block (stride or distance between each thread reduction) 
    stride_threads = 1
    # parallel reduction between blocks has a stride of 
    stride_blocks = (block - 1)*num_elements
    
    # shared mem to buffer the a elements
    shared = @cuStaticSharedMem(T, (2048,))
    @inbounds shared[thread] = a[thread + stride_blocks]
    @inbounds shared[thread + blockDim().x] = a[thread + stride_blocks + blockDim().x]
 
    # while still have elements to reduce 
    while stride_threads < num_elements
        # add a barrier to sync threads
        sync_threads()
        # compute index to reduce 
        index = 2*stride_threads*(thread - 1) + 1 
        # check index and index + d are inbounds a
        @inbounds if index ≤ num_elements && index + stride_threads + stride_blocks ≤ length(a)
            shared[index] = op(shared[index], shared[index + stride_threads])
        end
        stride_threads *= 2
    end
    # do attomic operatios with the first entry of ech block reduction at shared 
    if thread == 1 
        CUDA.@atomic b[] = op(b[], shared[1])
    end
    return nothing
end
# define test inputs
c_a = 1:16
d_a = CuArray(1:16)
d_b = CuArray([0])
# lunch kernel
@cuda(
    threads = 4,
    blocks = 2,
    reduce_grid_shared(+, d_a, d_b)
    )
# test the result 
using Test
CUDA.@allowscalar d_b
@test CUDA.@allowscalar d_b[] == sum(c_a)

Test Passed
  Expression: #= In[12]:46 =# CUDA.@allowscalar d_b[] == sum(c_a)

In [18]:
# Let's test it 
N = 2048
@benchmark CUDA.@sync @cuda( 
    threads = 1024, 
    blocks = 2,
        reduce_grid_shared(+,   $(CUDA.rand(N, N)), $(CUDA.rand(N, N)))
    )

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.584 μs … 120.274 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.307 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.144 μs ±   3.643 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅▆█▇▆▄▄▄▄▃▃▂▂▂▂▁▁▁▁                              ▁         ▂
  ▄███████████████████████████▇▇▇▆▆▆▅▅▅▆▄▅▄▃▄▄▄▄▄▄▄▆██▃▄▄▄▆▇█ █
  7.58 μs      Histogram: log(frequency) by time      17.4 μs <

 Memory estimate: 304 bytes, allocs estimate: 5.

### With new GPU's shared memory is not a big advantage in essence, since atomic operations to global memory are already optimezed. But we can use the sum operation and test it 

In [17]:
N = 2048
@benchmark CUDA.@sync CUDA.sum($(CUDA.rand(N, N)))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   86.281 μs …  44.019 ms  ┊ GC (min … max): 0.00% … 36.11%
 Time  (median):      97.918 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   107.170 μs ± 440.431 μs  ┊ GC (mean ± σ):  1.48% ±  0.36%

            ▆█▇▄▁                                                
  ▂▂▃▃▃▃▂▂▃███████▆▅▅▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂ ▃
  86.3 μs          Histogram: frequency by time          140 μs <

 Memory estimate: 4.67 KiB, allocs estimate: 90.

In [20]:
@benchmark sum(rand(N,N))

BenchmarkTools.Trial: 356 samples with 1 evaluation.
 Range (min … max):   8.171 ms … 22.544 ms  ┊ GC (min … max):  0.00% … 37.76%
 Time  (median):     13.681 ms              ┊ GC (median):    13.95%
 Time  (mean ± σ):   14.065 ms ±  3.451 ms  ┊ GC (mean ± σ):  18.43% ± 15.61%

    ▅    ▅▄▁▁   ▂▂█  █▂▁▁▃   ▂          ▆▄▁▅▇▅   ▃ ▁           
  ▅▆██▅▇▆██████████▇▄█████▅█▇██▆█▃▄▅▇▅█▅██████▇█▇█▇█▆▄▄▃▄▁▁▆▆ ▅
  8.17 ms         Histogram: frequency by time        20.9 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 3.